In [98]:
import pandas as pd
import numpy as np
from abc import ABC
from typing import Callable
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import math as ma
import random
import csv
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

***Função de Normalização dos Dados***

In [99]:
class NormalizationData():
  def z_score(arr: np.ndarray):
    mean = arr.mean()
    std = arr.std()
    out = (arr - mean) / std
    return out

***Importando Dados***

In [100]:
df = np.genfromtxt("kc2.csv", delimiter=",")
X = df[:,:-1]
y = df[:,-1]

***Normalização de dados com Z-score***

In [101]:
X_norm = np.empty_like(X)
for idx, col in enumerate(X.T):
  norm = NormalizationData.z_score(col)
  X_norm[:, idx] = norm

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=40, stratify=y)

***Métricas de avaliação***

In [103]:
class ClassificationMetrics():
  def accuracy(y, y_predito) -> float:
    return ((y == y_predito).sum() / y.shape[0])
      
  def precision(y, y_predito ) -> float:
    real = y.astype(bool)
    pred = y_predito.astype(bool)
    T = real == 1
    F = ~T
    TP = pred[T].sum()
    FP = (~pred[F]).sum()
    return (TP / (TP + FP))

  def recall(y, y_predito) -> float:
    real = y.astype(bool)
    pred = y_predito.astype(bool)
    T = real == True
    TP = pred[T].sum()
    FN = (~pred[T]).sum()
    return (TP / (TP + FN))
    
  def f1_score(y, y_predito) -> float:
    rec = ClassificationMetrics.recall(y, y_predito)
    prec = ClassificationMetrics.precision(y, y_predito)
    return (2 * (rec * prec) / (rec + prec))
   

***Questão 01***

**ITEM (A)**

***Funções de distância - Euclidiana e Mahalanobis***

In [104]:
class Euclidian():
  def dists(x1, x2) -> float:
    return np.sqrt(np.sum((x2 - x1) ** 2))

class Mahalanobis():
  def dists(x1, x2) -> float:
    inv_cov = np.linalg.pinv(np.cov(X_train.T))
    return np.sqrt((x1 - x2).T @ inv_cov @ (x1 - x2))

***K-ésimo Vizinho mais Próximo - KNN***

In [105]:
class KNN():
  def __init__(self, x, y, k, dist_func: Callable[[np.ndarray, np.ndarray], float]):
    self.x = x
    self.y = y
    self.k = k
    self.dist_func = dist_func
      
  def fit(self, X, y):
    self.X = X
    self.y = y

  def calculate_dist(self, rown: np.ndarray, x: np.ndarray) -> np.ndarray:
    return np.apply_along_axis(lambda x_rown: self.dist_func(rown, x_rown), 1, x)

  def k_nearest(self, dists: np.ndarray) -> np.ndarray:
    partition = np.argpartition(dists, self.k)
    result_id = partition[:self.k]
    return result_id

  def prediction(self, rown: np.ndarray) -> np.ndarray:
    dists_others_points = self.calculate_dist(rown, self.x)
    k_nearests_id = self.k_nearest(dists_others_points)
    class_points = self.y[k_nearests_id]
    unique_class, counts = np.unique(class_points, return_counts=True)
    most_freq_class_id = np.argmax(counts)
    return unique_class[most_freq_class_id]

  def predict(self, x: np.ndarray) -> np.ndarray:
    return np.apply_along_axis(self.prediction, 1, x).reshape([-1, 1])

**KNN**

***k=1 e Distância Euclidiana***


In [106]:
k = 1
knn_euclidian = KNN(X_train, y_train, k, Euclidian.dists)
knn_euclidian.fit(X_train, y_train)

preds_euclidian = knn_euclidian.predict(X_test)
accuracy = ClassificationMetrics.accuracy(y_test, preds_euclidian)
precision = ClassificationMetrics.precision(y_test, preds_euclidian)
recall = ClassificationMetrics.recall(y_test, preds_euclidian)
f1_score = ClassificationMetrics.f1_score(y_test, preds_euclidian)

print("KNN - k=1 | Resultado para as métricas \n")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1_score: ", f1_score)

KNN - k=1 | Resultado para as métricas 

Accuracy:  32.4
Precision:  0.5681818181818182
Recall:  0.78125
F1_score:  0.6578947368421052


**KNN**

***k=5 e Distância Euclidiana***


In [107]:
k = 5
knn_euclidian = KNN(X_train, y_train, k, Euclidian.dists)
knn_euclidian.fit(X_train, y_train)

preds_euclidian = knn_euclidian.predict(X_test)
accuracy = ClassificationMetrics.accuracy(y_test, preds_euclidian)
precision = ClassificationMetrics.precision(y_test, preds_euclidian)
recall = ClassificationMetrics.recall(y_test, preds_euclidian)
f1_score = ClassificationMetrics.f1_score(y_test, preds_euclidian)

print("KNN - k=5 | Resultado para as métricas \n")
print("Acurácia: ", accuracy)
print("Precisão: ", precision)
print("Revocação: ", recall)
print("F1-score: ", f1_score)

KNN - k=5 | Resultado para as métricas 

Acurácia:  32.43076923076923
Precisão:  0.5434782608695652
Revocação:  0.78125
F1-score:  0.641025641025641


**KNN**

***k=1 e Distância de Mahalanobis***


In [108]:
k = 1
knn_mahalanobis = KNN(X_train, y_train, k, Mahalanobis.dists )
knn_mahalanobis.fit(X_train, y_train)

preds_mahalanobis = knn_mahalanobis.predict(X_test)
accuracy = ClassificationMetrics.accuracy(y_test, preds_mahalanobis)
precision = ClassificationMetrics.precision(y_test, preds_mahalanobis)
recall = ClassificationMetrics.recall(y_test, preds_mahalanobis)
f1_score = ClassificationMetrics.f1_score(y_test, preds_mahalanobis)

print("KNN - k=1 | Resultado para as métricas \n")
print("Acurácia: ", accuracy)
print("Precisão: ", precision)
print("Revocação: ", recall)
print("F1-score: ", f1_score)

KNN - k=1 | Resultado para as métricas 

Acurácia:  32.46153846153846
Precisão:  0.5238095238095238
Revocação:  0.6875
F1-score:  0.5945945945945946


**KNN**

***k=5 e Distância de Mahalanobis***


In [109]:
k = 5
knn_mahalanobis = KNN(X_train, y_train, k, Mahalanobis.dists )
knn_mahalanobis.fit(X_train, y_train)

preds_mahalanobis = knn_mahalanobis.predict(X_test)
accuracy = ClassificationMetrics.accuracy(y_test, preds_mahalanobis)
precision = ClassificationMetrics.precision(y_test, preds_mahalanobis)
recall = ClassificationMetrics.recall(y_test, preds_mahalanobis)
f1_score = ClassificationMetrics.f1_score(y_test, preds_mahalanobis)

print("KNN - k=5 | Resultado para as métricas \n")
print("Acurácia: ", accuracy)
print("Precisão: ", precision)
print("Revocação: ", recall)
print("F1-score: ", f1_score)

KNN - k=5 | Resultado para as métricas 

Acurácia:  32.55384615384615
Precisão:  0.4583333333333333
Revocação:  0.6875
F1-score:  0.5499999999999999


***Árvore de Decisão***

In [110]:
dt = DecisionTreeClassifier(criterion='entropy')
dt = dt.fit(X_train, y_train)
prediction = dt.predict(X_test)

accuracy = ClassificationMetrics.accuracy(y_test, prediction)
precision = ClassificationMetrics.precision(y_test, prediction)
recall = ClassificationMetrics.recall(y_test, prediction)
f1_score = ClassificationMetrics.f1_score(y_test, prediction)

print("Árvore de Decisão | Resultado para as métricas \n")
print("Acurácia: ", accuracy)
print("Pecisão: ", precision)
print("Revocação: ", recall)
print("F1-Score: ", f1_score)

Árvore de Decisão | Resultado para as métricas 

Acurácia:  0.7230769230769231
Pecisão:  0.5531914893617021
Revocação:  0.8125
F1-Score:  0.6582278481012658


**ITEM (B)**

Para cada modelo criado, reporte valor médio e desvio padrão das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**

In [111]:
class Kfolds():
  def __init__(self, folds = 10):
    self.folds = folds
      
  def fold(self):
    np.random.shuffle(self.data)
    n = self.data.shape[0]
    self.n_folds = []
    folds_size = ma.ceil(n / self.folds)     
    for i in range(0, n, folds_size):
      end = i + folds_size
      end = n if end > n else end
      fold = self.data[i: end,:]
      self.n_folds.append(fold)
            
  def predict(self, index):
    folds = self.n_folds[:]
    test_fold = folds.pop(index)
    train_fold = np.vstack(folds)
    x_train = train_fold[:, :-1]
    y_train = train_fold[:, -1]
    x_test = test_fold[:, :-1]
    y_test = test_fold[:, -1]
    self.model.fit(x_train, y_train)
    pred = self.model.predict(x_test)
    return pred, y_test
    
  def folds_results_metrics(self):
    self.accuracies = []
    self.precisions = []
    self.recalls = []
    self.f1_score = []
    
    for i in range(self.folds):
      pred, y_test = self.predict(i)
      accuracy = ClassificationMetrics.accuracy(y_test, pred)
      precision = ClassificationMetrics.precision(y_test, pred)
      recall = ClassificationMetrics.recall(y_test, pred)
      f1_score = ClassificationMetrics.f1_score(y_test, pred)
        
    self.accuracies.append(accuracy)
    self.precisions.append(precision)
    self.recalls.append(recall)
    self.f1_score.append(f1_score)
          
    self.accurancy_mean = np.mean(self.accuracies)
    self.precision_mean = np.mean(self.precisions)
    self.recall_mean = np.mean(self.recalls)
    self.f1_score_mean = np.mean(self.f1_score)
      
    self.accurancy_std = np.std(self.accuracies)
    self.precision_std = np.std(self.precisions)
    self.recall_std = np.std(self.recalls)
    self.f1_score_std = np.std(self.f1_score)
        
  def fit_folds(self, model, x, y):
    self.data = np.c_[ x, y ]
    self.model = model
    self.fold()
    self.folds_results_metrics()

In [112]:
models = {
    'K-NN com distancia euclidiana ': knn_euclidian,
    'K-NN com distancia de mahalanobis': knn_mahalanobis,
    'Árvore de Decisão' : dt
}
models_names = list(models.keys())

In [113]:
accuracies_means = []
precisions_means = []
f1_score_means = []
recalls_means = []

accuracies_stds = []
precisions_stds = []
f1_score_stds = []
recalls_stds = []

for model_name, model in models.items():
  kfolds = Kfolds()
  kfolds.fit_folds(model, X, y)

  accuracies_means.append(kfolds.accurancy_mean)
  precisions_means.append(kfolds.precision_mean)
  recalls_means.append(kfolds.recall_mean)
  f1_score_means.append(kfolds.f1_score_mean)

  accuracies_stds.append(kfolds.accurancy_std)
  precisions_stds.append(kfolds.precision_std)
  recalls_stds.append(kfolds.recall_std)
  f1_score_stds.append(kfolds.f1_score_std)
  

***Resultados das métricas com K-fold***

In [114]:
metrics = ["Acurácia", "Precisão", "Revocação", "F1-Score"]
dt_metrics_mean, knn_eucli_metrics_mean, knn_mahal_metrics_mean = np.array([accuracies_means, precisions_means, recalls_means, f1_score_means]).T
dt_metrics_std, knn_eucli_metrics_std, knn_mahal_metrics_std = np.array([accuracies_stds, precisions_stds, recalls_stds, f1_score_stds]).T

***Média das métricas***

***KNN - distancia euclidiana***

In [115]:
print("Resultado com K-folds")
print("Média - KNN com distancia euclidiana\n")

print("Acurácia do modelo:", knn_eucli_metrics_mean[0])
print("Precisão do modelo:", knn_eucli_metrics_mean[1])
print("Revocação do modelo:", knn_eucli_metrics_mean[2])
print("F1-score do modelo:", knn_eucli_metrics_mean[3])

Resultado com K-folds
Média - KNN com distancia euclidiana

Acurácia do modelo: 8.0
Precisão do modelo: 0.2
Revocação do modelo: 0.125
F1-score do modelo: 0.15384615384615385


***KNN - distancia de mahalanobis***

In [116]:
print("Resultado com K-folds")
print("Média - KNN com distancia de mahalanobis \n")

print("Acurácia do modelo:", knn_mahal_metrics_mean[0])
print("Precisão do modelo:", knn_mahal_metrics_mean[1])
print("Revocação do modelo:", knn_mahal_metrics_mean[2])
print("F1-score do modelo:", knn_mahal_metrics_mean[3])

Resultado com K-folds
Média - KNN com distancia de mahalanobis 

Acurácia do modelo: 0.6875
Precisão do modelo: 0.45454545454545453
Revocação do modelo: 0.5555555555555556
F1-score do modelo: 0.5


***Árvore de decisão***

In [117]:
print("Resultado com K-folds")
print("Média - Árvore de decisão  \n")

print("Acurácia do modelo:", dt_metrics_mean[0])
print("Precisão do modelo:", dt_metrics_mean[1])
print("Revocação do modelo:", dt_metrics_mean[2])
print("F1-score do modelo:", dt_metrics_mean[3])

Resultado com K-folds
Média - Árvore de decisão  

Acurácia do modelo: 7.375
Precisão do modelo: 0.16666666666666666
Revocação do modelo: 0.1111111111111111
F1-score do modelo: 0.13333333333333333


***Desvio padrão das métricas***

***KNN - distancia euclidiana***



In [118]:
print("Resultado com K-folds")
print("Desvio padrão - KNN com distancia euclidiana\n")

print("Acurácia do modelo:", knn_eucli_metrics_std[0])
print("Precisão do modelo:", knn_eucli_metrics_std[1])
print("Revocação do modelo:", knn_eucli_metrics_std[2])
print("F1-score do modelo:", knn_eucli_metrics_std[3])

Resultado com K-folds
Desvio padrão - KNN com distancia euclidiana

Acurácia do modelo: 0.0
Precisão do modelo: 0.0
Revocação do modelo: 0.0
F1-score do modelo: 0.0


***KNN - distancia de mahalanobis***

In [119]:
print("Resultado com K-folds")
print("Desvio padrão - KNN com distancia de mahalanobis \n")

print("Acurácia do modelo: ", knn_mahal_metrics_std[0])
print("Precisão do modelo: ", knn_mahal_metrics_std[1])
print("Revocação do modelo: ", knn_mahal_metrics_std[2])
print("F1-score do modelo: ", knn_mahal_metrics_std[3])

Resultado com K-folds
Desvio padrão - KNN com distancia de mahalanobis 

Acurácia do modelo:  0.0
Precisão do modelo:  0.0
Revocação do modelo:  0.0
F1-score do modelo:  0.0


***Árvore de decisão***

In [120]:
print("Resultado com K-folds")
print("Desvio padrão - Árvore de decisão  \n")

print("Acurácia do modelo: ", dt_metrics_std[0])
print("Precisão do modelo: ", dt_metrics_std[1])
print("Revocação do modelo: ", dt_metrics_std[2])
print("F1-score do modelo: ", dt_metrics_std[3])

Resultado com K-folds
Desvio padrão - Árvore de decisão  

Acurácia do modelo:  0.0
Precisão do modelo:  0.0
Revocação do modelo:  0.0
F1-score do modelo:  0.0
